In [1]:
pip install biopython requests pandas


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


### 데이터 제공 방법
1. PMC article datasets
2. 2.PMC Journal List

# 1. PMC article datasets

In [4]:
from Bio import Entrez
import time

Entrez.email = "parksungyong0101@naver.com" # <- 본인 이메일

# 키워드로 PMC 논문 검색
def search_pmc(keyword, max_results=100):
    handle = Entrez.esearch(db="pmc", term=keyword, retmax=max_results)
    record = Entrez.read(handle)
    handle.close()
    return record['IdList']

# 논문 XML 다운로드
def fetch_pmc(pmc_id):
    handle = Entrez.efetch(db="pmc", id=pmc_id, rettype="full", retmode="xml")
    xml = handle.read()
    handle.close()
    return xml

search_keyword = "Drug Interaction"  # <-- 검색할 키워드
max_download_articles = 10           # <-- 다운로드할 논문 수 (필수입력)

# 예시 사용법
pmc_ids = search_pmc("Drug Interaction", max_results=10)

for pmc_id in pmc_ids:
    xml_data = fetch_pmc(pmc_id).decode('utf-8')
    with open(f"{pmc_id}.xml", "w", encoding="utf-8") as f:
        f.write(xml_data)
    time.sleep(0.5)  # 과도한 요청 방지

In [11]:
from Bio import Entrez
import pandas as pd
import time

# 본인 이메일 필수 입력!
Entrez.email = "your_email@example.com"  # <-- 필수 수정!

def search_pmc(keyword, max_results=10):
    query = f'("{keyword}"[Title] OR "{keyword}"[Abstract])'
    handle = Entrez.esearch(db="pmc", term=query, retmax=max_results)
    record = Entrez.read(handle)
    handle.close()
    return record['IdList']

def fetch_pmc_metadata(pmc_id):
    handle = Entrez.efetch(db="pmc", id=pmc_id, retmode="xml")
    xml_data = handle.read()
    handle.close()
    return xml_data

def parse_pmc_xml(xml_data):
    from bs4 import BeautifulSoup
    soup = BeautifulSoup(xml_data, 'lxml')

    title_tag = soup.find('article-title')
    title = title_tag.get_text(strip=True) if title_tag else ""

    abstract_tags = soup.find_all('abstract')
    abstracts = ' '.join(ab.get_text(strip=True) for ab in abstract_tags)

    return title, abstracts

# 사용자 입력 부분
search_keyword = "drug interaction"  # <-- 검색 키워드 입력!
max_articles = 10                         # <-- 논문 최대 개수 입력!

pmc_ids = search_pmc(search_keyword, max_results=max_articles)

data = []
for pmc_id in pmc_ids:
    xml_data = fetch_pmc_metadata(pmc_id)
    title, abstract = parse_pmc_xml(xml_data)
    data.append({
        'PMC_ID': pmc_id,
        'Title': title,
        'Abstract': abstract
    })
    print(f"{pmc_id} 완료")
    time.sleep(0.5)

# CSV로 저장하기
df = pd.DataFrame(data)
df.to_csv("pmc_articles.csv", index=False, encoding='utf-8')

print("CSV 저장 완료: pmc_articles.csv")

11946286 완료
11935471 완료
11920219 완료
11919272 완료
11910312 완료
11903327 완료
11886857 완료
11893426 완료
11869883 완료
11879991 완료
CSV 저장 완료: pmc_articles.csv
